# <span style="background-color:white;color:red;"> ⌛⌛ EN PROGRESO⌛⌛</span>

## Bloque I
Descomprimir el csv en formato gz

In [284]:
"""  
Importación de ficheros de AIRBNB. Preprocesado y cargado a la base de datos 

Obtenemos los datos desde https://insideairbnb.com/get-the-data/

Podríamos hacer scraping, obtener la fecha de la última actualización de Madrid y generar el enlace para descargarlo, pero no es necesario por que se actualiza cada mucho tiempo y no es necesario tener la última versión. Podemos cambiar la info procesable en la carpeta data manualmente.

"""
import os
import pandas as pd
import numpy as np
import datetime
#from config_bd import *
import requests
import sys
import gzip
import shutil


In [285]:

# DATA_DIR = os.path.join('data')
# # DATA_DIR = os.path.join(os.path.dirname(__file__), 'data')
# TEMP_DIR = os.path.join(DATA_DIR, 'TEMP')


# """ Comprobamos que existe el directorio de datos """

# if not os.path.exists(DATA_DIR):
#     print(f"El directorio {DATA_DIR} no existe. No es posible continuar...")
#     sys.exit()

# """ Descomprimimos el archivo de calendario """

# if not os.path.exists(TEMP_DIR):
#     os.makedirs(TEMP_DIR)


# gz_file_path = os.path.join(DATA_DIR, 'listings.csv.gz')
# output_file_path = os.path.join(TEMP_DIR, os.path.splitext(os.path.basename(gz_file_path))[0])

# if os.path.exists(gz_file_path):
#     try:
#         with gzip.open(gz_file_path, 'rb') as f_in:
#             with open(output_file_path, 'wb') as f_out:
#                 shutil.copyfileobj(f_in, f_out)
#         print(f"Archivo {gz_file_path} descomprimido como {output_file_path}")
#     except Exception as e:
#         print(f"Error al descomprimir el archivo {gz_file_path}: {e}")
#         sys.exit()
# else:
#     print(f"El archivo {gz_file_path} no existe. No es posible continuar...")
#     sys.exit()


## Bloque II
Carga del csv de detalle completo en pandas. Cambios de tipos e imputación

Estos son lo pasos seguidos:

<ul>
<li>Eliminamos campos innecesarios</li>
<li>Eliminamos las propiedades que no se encuentran dentro del perímetro de interés (habitaciones, compartidas, albergues...)</li>
<li></li>
<li></li>
<li></li>
<li></li>
<li></li>
</ul>

In [286]:
output_file_path = './data/listings.csv'

In [287]:
df = pd.read_csv(output_file_path, low_memory=False)

In [288]:
df
df['aire'] = df['amenities'].apply(lambda x: 1 if '"Air conditioning"' in x else 0)
df['garaje'] = df['amenities'].apply(lambda x: 1 if 'free parking' in x.lower() else 0)
df['calefaccion'] = df['amenities'].apply(lambda x: 1 if '"Heating"' in x else 0)
df['ascensor'] = df['amenities'].apply(lambda x: 1 if '"Elevator"' in x else 0)

In [289]:
df

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,aire,garaje,calefaccion,ascensor
0,21853,https://www.airbnb.com/rooms/21853,20250305023340,2025-03-11,city scrape,Bright and airy room,We have a quiet and sunny room with a good vie...,We live in a leafy neighbourhood with plenty o...,https://a0.muscache.com/pictures/68483181/87bc...,83531,...,f,2,0,2,0,0.26,1,1,1,1
1,30320,https://www.airbnb.com/rooms/30320,20250305023340,2025-03-10,previous scrape,Great Vacational Apartments,NaN,NaN,https://a0.muscache.com/pictures/336868/f67409...,130907,...,f,3,3,0,0,0.96,1,0,1,1
2,30959,https://www.airbnb.com/rooms/30959,20250305023340,2025-03-10,previous scrape,Beautiful loft in Madrid Center,Beautiful Loft 60m2 size just in the historica...,NaN,https://a0.muscache.com/pictures/78173471/835e...,132883,...,f,1,1,0,0,0.07,0,0,1,1
3,40916,https://www.airbnb.com/rooms/40916,20250305023340,2025-03-10,previous scrape,Holiday Apartment Madrid Center,NaN,NaN,https://a0.muscache.com/pictures/336736/c3b486...,130907,...,f,3,3,0,0,0.28,1,0,1,1
4,62423,https://www.airbnb.com/rooms/62423,20250305023340,2025-03-12,city scrape,MAGIC ARTISTIC HOUSE IN THE CENTER OF MADRID,INCREDIBLE HOME OF AN ARTIST SURROUNDED BY PAI...,DISTRICT WITH VERY GOOD VIBES IN THE MIDDLE OF...,https://a0.muscache.com/pictures/miso/Hosting-...,303845,...,f,3,1,2,0,2.73,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25283,1368985680426539467,https://www.airbnb.com/rooms/1368985680426539467,20250305023340,2025-03-12,city scrape,Acogedor y nuevo ático con gran terraza (B),"Cozy penthouse floor with large terrace, locat...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,20803102,...,f,3,3,0,0,NaN,1,0,1,1
25284,1368986418960986344,https://www.airbnb.com/rooms/1368986418960986344,20250305023340,2025-03-05,city scrape,Elegante apartamento en Madrid!,Have fun with the whole family at this stylish...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,52530675,...,f,45,45,0,0,NaN,1,0,0,0
25285,1368991577357063431,https://www.airbnb.com/rooms/1368991577357063431,20250305023340,2025-03-10,city scrape,Tropical apartment Madrid,Get away from routine at this unique and relax...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,52530675,...,f,45,45,0,0,NaN,1,0,0,0
25286,1369034696848967860,https://www.airbnb.com/rooms/1369034696848967860,20250305023340,2025-03-07,city scrape,"Cama 1 de 04, no es hinchable.",Enjoy the comfort and tranquility of this home.,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,543251626,...,f,4,0,0,4,NaN,0,0,0,0


In [290]:
df[df['garaje']==1]

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,aire,garaje,calefaccion,ascensor
0,21853,https://www.airbnb.com/rooms/21853,20250305023340,2025-03-11,city scrape,Bright and airy room,We have a quiet and sunny room with a good vie...,We live in a leafy neighbourhood with plenty o...,https://a0.muscache.com/pictures/68483181/87bc...,83531,...,f,2,0,2,0,0.26,1,1,1,1
20,100411,https://www.airbnb.com/rooms/100411,20250305023340,2025-03-10,previous scrape,Ático muy luminoso,Penthouse with a beautiful terrace and expansi...,"Very well connected residential area, with a l...",https://a0.muscache.com/pictures/673487/3c41ea...,527760,...,f,1,1,0,0,0.03,0,1,1,1
23,136058,https://www.airbnb.com/rooms/136058,20250305023340,2025-03-05,city scrape,"Calm, affordable room in Madrid.","The apartment is located in a safe, quiet and ...","The neighborhood is quiet, beautiful, safe and...",https://a0.muscache.com/pictures/85a31ad7-2224...,666482,...,f,1,0,1,0,0.58,0,1,1,1
25,167183,https://www.airbnb.com/rooms/167183,20250305023340,2025-03-12,city scrape,private house B & B. Arturo Soria (Metro),"Residential area, very quite house, no more gu...","The best location in Arturo Soria, next to Par...",https://a0.muscache.com/pictures/3458309/73f62...,796746,...,f,2,0,2,0,0.29,1,1,1,0
38,689099,https://www.airbnb.com/rooms/689099,20250305023340,2025-03-07,previous scrape,Lujo a tu alcance,"A quiet room, without the crowds of downtown, ...",A very quiet area. We have several cafes-resta...,https://a0.muscache.com/pictures/54356813-b2d7...,3357259,...,f,1,0,1,0,0.10,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25238,1368304377290123314,https://www.airbnb.com/rooms/1368304377290123314,20250305023340,2025-03-11,city scrape,Cómoda habitación doble,Comfortable room with 150 cm bed and built-in ...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,368390948,...,f,1,0,1,0,1.00,1,1,1,1
25239,1368305303631228330,https://www.airbnb.com/rooms/1368305303631228330,20250305023340,2025-03-05,city scrape,"Habitación, Ensanche de Vallecas",Quiet accommodation to spend your days in Madr...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,416541176,...,f,1,0,1,0,NaN,1,1,0,0
25240,1368313760167698062,https://www.airbnb.com/rooms/1368313760167698062,20250305023340,2025-03-12,city scrape,"Chamartín, Vive Madrid.",Welcome to your home away from home!<br />This...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,681979786,...,t,1,1,0,0,NaN,1,1,0,0
25253,1366584727244034857,https://www.airbnb.com/rooms/1366584727244034857,20250305023340,2025-03-12,city scrape,"2. Estudio con Baño,Cocina privada y Aire acon...","Private studio. With a private bathroom, washi...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,490812264,...,t,17,17,0,0,NaN,1,1,0,0


Eliminamos campos innecesarios

In [291]:
df = df.drop(columns=['scrape_id','last_scraped','source','neighborhood_overview','host_id','host_url','host_name','host_since','host_location','host_response_time','host_response_rate','host_acceptance_rate','host_is_superhost','host_thumbnail_url','host_picture_url','host_listings_count','host_total_listings_count','host_verifications','host_has_profile_pic','host_identity_verified','neighbourhood','bathrooms_text','beds','minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm','has_availability','calendar_last_scraped','number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d','availability_eoy','number_of_reviews_ly','first_review','last_review','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','license','instant_bookable','calculated_host_listings_count','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms','reviews_per_month','host_about','host_neighbourhood','calendar_updated','availability_30','availability_60','availability_90','availability_365','picture_url'])

Eliminamos las habitaciones, casas compartidas, hoteles, picaderos, etc, solo queremos viviendas completas

In [292]:
# df['property_type'].value_counts()
# df['room_type'].value_counts()
df = df[df['room_type']== 'Entire home/apt'].copy()
df['room_type'].value_counts()

room_type
Entire home/apt    16805
Name: count, dtype: int64

conversión de tipos

In [293]:
df['accommodates'] = df['accommodates'].astype(int)
df['bathrooms'] = df['bathrooms'].replace([np.inf,-np.inf],np.nan).fillna(0).astype(int)
df['bedrooms'] = df['bedrooms'].replace([np.inf,-np.inf],np.nan).fillna(0).astype(int)
df['price'] = df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df['minimum_nights'] = df['minimum_nights'].astype(int)
df['maximum_nights'] = df['maximum_nights'].astype(int)
df['estimated_occupancy_l365d'] = df['estimated_occupancy_l365d'].astype(int)

imputamos una habitación y un baño cuando esta información no está rellena, asumiendo que habiendo retirado las propiedades compartidas, podemos interpretar que los estudios, lofts, etc todos van a tener al menos una habitación y baño o equivalente. (Sofá cama, etc).

*También se podría imputar las habitaciones dividiendo los "acommodates" entre dos pero creo que este método es más conservador.*

In [294]:
df

,id,listing_url,name,description,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,...,amenities,price,minimum_nights,maximum_nights,estimated_occupancy_l365d,estimated_revenue_l365d,aire,garaje,calefaccion,ascensor
1,30320,https://www.airbnb.com/rooms/30320,Great Vacational Apartments,NaN,Sol,Centro,40.414760,-3.704180,Entire rental unit,Entire home/apt,...,"[""Kitchen"", ""Air conditioning"", ""Elevator"", ""T...",NaN,5,180,0,NaN,1,0,1,1
2,30959,https://www.airbnb.com/rooms/30959,Beautiful loft in Madrid Center,Beautiful Loft 60m2 size just in the historica...,Embajadores,Centro,40.412590,-3.701050,Entire loft,Entire home/apt,...,"[""Shampoo"", ""Smoking allowed"", ""Pets allowed"",...",NaN,3,730,0,NaN,0,0,1,1
3,40916,https://www.airbnb.com/rooms/40916,Holiday Apartment Madrid Center,NaN,Universidad,Centro,40.422470,-3.705770,Entire rental unit,Entire home/apt,...,"[""Pets allowed"", ""Kitchen"", ""TV"", ""Air conditi...",NaN,5,180,0,NaN,1,0,1,1
5,70059,https://www.airbnb.com/rooms/70059,Tu hogar en centro de Madrid.,"Very nice, cozy and bright, right in the cente...",Universidad,Centro,40.423810,-3.710380,Entire rental unit,Entire home/apt,...,"[""Paid parking off premises"", ""Window AC unit""...",66.0,15,365,0,0.0,0,0,1,1
6,70310,https://www.airbnb.com/rooms/70310,"Heart of Malasaña Cozy, Quiet & Sunny Apartment",Cozy Apartment in great location in the center...,Universidad,Centro,40.422520,-3.702500,Entire rental unit,Entire home/apt,...,"[""Portable fans"", ""Paid parking off premises"",...",89.0,30,360,0,0.0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25281,1368974630547284843,https://www.airbnb.com/rooms/1368974630547284843,Coqueto estudio en tetuan!,Disconnect from the routine in this charming a...,Valdeacederas,Tetuán,40.464951,-3.704578,Entire rental unit,Entire home/apt,...,"[""Air conditioning"", ""Pets allowed"", ""Wifi"", ""...",67.0,5,365,0,0.0,1,0,0,0
25282,1368981094706864086,https://www.airbnb.com/rooms/1368981094706864086,Bonito estudio en Tetuán!,Get away from routine at this unique and relax...,Valdeacederas,Tetuán,40.468355,-3.706939,Entire rental unit,Entire home/apt,...,"[""Air conditioning"", ""Pets allowed"", ""Wifi"", ""...",63.0,5,365,0,0.0,1,0,0,0
25283,1368985680426539467,https://www.airbnb.com/rooms/1368985680426539467,Acogedor y nuevo ático con gran terraza (B),"Cozy penthouse floor with large terrace, locat...",Canillas,Hortaleza,40.464110,-3.647130,Entire rental unit,Entire home/apt,...,"[""Cleaning available during stay"", ""Wifi"", ""Wi...",103.0,2,365,0,0.0,1,0,1,1
25284,1368986418960986344,https://www.airbnb.com/rooms/1368986418960986344,Elegante apartamento en Madrid!,Have fun with the whole family at this stylish...,San Isidro,Carabanchel,40.395983,-3.730953,Entire rental unit,Entire home/apt,...,"[""Pets allowed"", ""Kitchen"", ""TV"", ""Air conditi...",100.0,5,365,0,0.0,1,0,0,0


In [295]:
# df[(df['bathrooms'] == 0)& (df['bedrooms']==0)]
df['bathrooms'] = df['bathrooms'].replace(0,1)
df['bedrooms'] = df['bedrooms'].replace(0,1)

In [296]:
df = df.rename(columns={'id':'property_id'})

In [297]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16805 entries, 1 to 25285
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   property_id                   16805 non-null  int64  
 1   listing_url                   16805 non-null  object 
 2   name                          16805 non-null  object 
 3   description                   16344 non-null  object 
 4   neighbourhood_cleansed        16805 non-null  object 
 5   neighbourhood_group_cleansed  16805 non-null  object 
 6   latitude                      16805 non-null  float64
 7   longitude                     16805 non-null  float64
 8   property_type                 16805 non-null  object 
 9   room_type                     16805 non-null  object 
 10  accommodates                  16805 non-null  int64  
 11  bathrooms                     16805 non-null  int64  
 12  bedrooms                      16805 non-null  int64  
 13  amenit

## Bloque III
Importamos a la base de datos. 
**Requiere haber creado la tabla** <code>01_CREATE TABLE bnb_property.sql</code>

In [298]:
#from config_bd import get_connection

In [299]:
#df.to_csv(os.path.join(TEMP_DIR,'transform_listings.csv'), index=False)

In [300]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16805 entries, 1 to 25285
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   property_id                   16805 non-null  int64  
 1   listing_url                   16805 non-null  object 
 2   name                          16805 non-null  object 
 3   description                   16344 non-null  object 
 4   neighbourhood_cleansed        16805 non-null  object 
 5   neighbourhood_group_cleansed  16805 non-null  object 
 6   latitude                      16805 non-null  float64
 7   longitude                     16805 non-null  float64
 8   property_type                 16805 non-null  object 
 9   room_type                     16805 non-null  object 
 10  accommodates                  16805 non-null  int64  
 11  bathrooms                     16805 non-null  int64  
 12  bedrooms                      16805 non-null  int64  
 13  amenit

### Amenities

In [301]:
import ast

def get_all_amenities(data):
    """
    Devuelve un set con todos los amenities únicos presentes en la columna 'amenities' del DataFrame.
    """
    all_amenities = set()
    for amenities_str in data["amenities"]:
        try:
            amenities_list = ast.literal_eval(amenities_str)
            all_amenities.update(amenities_list)
        except Exception:
            continue
    return all_amenities

In [302]:
amenities = get_all_amenities(df)

In [303]:
len(amenities)

2883

In [304]:
# df['aire'] = df['amenities'].apply(lambda x: 1 if '"Air conditioning"' in x else 0)
# df['garaje'] = df['amenities'].apply(lambda x: 1 if 'free parking' in x.lower() else 0)
# df['ascensor'] = df['amenities'].apply(lambda x: 1 if '"Elevator"' in x else 0)

In [305]:
amentities_exclude = ['condition','pool','BBQ','sauna','conditioner','Exercise equipment','Paid Parking','sound system','Housekeeping','book','amazon','soap','shampoo','hair dryer','iron','washer','dryer','TV','internet','wifi','kitchen','microwave','refrigerator','stove','oven','dishwasher','coffee maker','toaster']

In [306]:
for exclude_amenity in amentities_exclude:
     amenities = [value for value in amenities if str.lower(exclude_amenity) not in str.lower(value)]


In [307]:
len(amenities)

246

In [308]:
amenities

['Lake access',
 'Hammock',
 'Paid folding or convertible high chair - available upon request',
 'Clothing storage: closet and wardrobe',
 'Shared hot tub',
 'Resort access',
 'Blender',
 'Heating - split type ductless system',
 'Free driveway parking on premises',
 'Roundtrip airport transfer – included with your stay',
 'Shared backyard – Fully fenced',
 'Paid crib',
 'Single level home',
 'Outlet covers',
 'Window guards',
 'Backyard - Not fully fenced',
 'Crib - available upon request',
 'Safe',
 'Indoor fireplace: wood-burning',
 'EV charger',
 'Crib - always at the listing',
 'Breakfast',
 'Free parking on premises – 1 space',
 'Indoor fireplace: ethanol',
 'Clothing storage: walk-in closet and wardrobe',
 'Outdoor shower',
 'Paid standalone high chair - available upon request',
 'Movie theater',
 'Free residential garage on premises',
 'Hockey rink',
 'Booster seat high chair - always at the listing',
 'Private patio or balcony',
 'Skate ramp',
 'Piano',
 'Paid high chair - avai

##### Heating

In [309]:
[value for value in amenities if 'heating' in str.lower(value)]

['Heating - split type ductless system',
 'Heating',
 'Central heating',
 'Radiant heating']

In [310]:
df['calefaccion'] = df['amenities'].apply(lambda x: 1 if 'heating' in x.lower() else 0)

In [311]:
df['calefaccion'].value_counts()

calefaccion
1    14473
0     2332
Name: count, dtype: int64

##### Pool

In [312]:
[value for value in amenities if ('pool' in str.lower(value)) and ('whirlpool' not in str.lower(value)) and ('whirpool' not in str.lower(value))]

[]

- Pool

In [313]:
df['pool'] = df['amenities'].apply(lambda x: 1 if ('pool' in str.lower(x)) and ('whirlpool' not in str.lower(x)) and ('whirpool' not in str.lower(x)) else 0)

In [314]:
df['pool'].value_counts()

pool
0    16047
1      758
Name: count, dtype: int64

##### Parking

In [315]:
[value for value in amenities if 'free parking' in str.lower(value)]

['Free parking on premises – 1 space',
 'Free parking garage on premises',
 'Free parking garage on premises – 51 spaces',
 'Free parking garage on premises – 2 spaces',
 'Free parking on premises',
 'Free parking garage on premises – 81 spaces',
 'Free parking garage on premises – 45 spaces',
 'Free parking garage on premises – 1 space',
 'Free parking garage on premises – 17 spaces']

In [316]:
df['garaje'] = df['amenities'].apply(lambda x: 1 if 'free parking' in x.lower() else 0)

In [317]:
df['garaje'].value_counts()

garaje
0    15856
1      949
Name: count, dtype: int64

##### Air conditioning

In [318]:
[value for value in amenities if 'air conditi' in str.lower(value)]

[]

In [319]:
df['aire_acondicionado'] = df['amenities'].apply(lambda x: 1 if 'air conditi' in str.lower(x) else 0)

In [320]:
df['aire_acondicionado'].value_counts()

aire_acondicionado
1    12296
0     4509
Name: count, dtype: int64

##### Elevator

In [321]:
[value for value in amenities if 'elevator' in str.lower(value)]

['Elevator']

In [322]:
df['ascensor'] = df['amenities'].apply(lambda x: 1 if 'elevator' in str.lower(x) else 0)

In [323]:
df['ascensor'].value_counts()

ascensor
0    8643
1    8162
Name: count, dtype: int64

##### Movilidad reducida

In [332]:
[value for value in amenities if 'single level home' in str.lower(value)]

['Single level home']

In [333]:
df['movilidad_reducida'] = df['amenities'].apply(lambda x: 1 if 'single level home' in str.lower(x) else 0)

In [ ]:
df['movilidad_reducida'].value_counts()

movilidad_reducida
0    14921
1     1884
Name: count, dtype: int64

In [ ]:
#OJO, no tiene sentido no tener ascensor y movilidad reducida
df[df['movilidad_reducida'] == 1][["listing_url","movilidad_reducida","ascensor"]]

,listing_url,movilidad_reducida,ascensor
9,https://www.airbnb.com/rooms/84719,1,1
37,https://www.airbnb.com/rooms/684564,1,1
72,https://www.airbnb.com/rooms/280733,1,1
78,https://www.airbnb.com/rooms/751527,1,1
80,https://www.airbnb.com/rooms/752424,1,1
...,...,...,...
25131,https://www.airbnb.com/rooms/1362319075445033257,1,0
25135,https://www.airbnb.com/rooms/1366625139989288134,1,1
25188,https://www.airbnb.com/rooms/1367400865560226670,1,1
25200,https://www.airbnb.com/rooms/1365826061924320510,1,0


#### Otros

In [324]:
df["name_description"] = df["name"].fillna(" ").str.lower() + " and " + df["description"].fillna(" ").str.lower()
df["name_description"].isnull().sum()


0

##### Terrace

In [325]:
df["terraza"] = df["name_description"].apply(lambda x: 1 if "terrace" in x else 0)

In [326]:
df["terraza"].value_counts()

terraza
0    15430
1     1375
Name: count, dtype: int64

##### Balconies

In [327]:
df["balcon"] = df["name_description"].apply(lambda x: 1 if "balcon" in x else 0)

In [328]:
df["balcon"].value_counts()

balcon
0    15759
1     1046
Name: count, dtype: int64

In [329]:
df[ (df['description'].str.contains('terrace', na=False))]

,property_id,listing_url,name,description,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,...,estimated_revenue_l365d,aire,garaje,calefaccion,ascensor,pool,aire_acondicionado,name_description,terraza,balcon
7,72150,https://www.airbnb.com/rooms/72150,Sunny attic duplex flat with terrace next to Sol,"Nestled in the serene heart of Madrid, this re...",Embajadores,Centro,40.411500,-3.704490,Entire rental unit,Entire home/apt,...,17646.0,1,0,1,0,0,1,sunny attic duplex flat with terrace next to s...,1,0
10,86042,https://www.airbnb.com/rooms/86042,Cozy attic with intimate rooftop terrace+ elev...,"Nestled in Madrid's tranquil heart, this apart...",Embajadores,Centro,40.409390,-3.698120,Entire rental unit,Entire home/apt,...,27216.0,1,0,1,1,0,1,cozy attic with intimate rooftop terrace+ elev...,1,0
20,100411,https://www.airbnb.com/rooms/100411,Ático muy luminoso,Penthouse with a beautiful terrace and expansi...,Valverde,Fuencarral - El Pardo,40.500440,-3.670320,Entire rental unit,Entire home/apt,...,NaN,0,1,1,1,1,0,ático muy luminoso and penthouse with a beauti...,1,0
27,198296,https://www.airbnb.com/rooms/198296,Very quiet central top floor with 2 terraces,Beautiful central top floor apartment with two...,Universidad,Centro,40.425890,-3.705650,Entire rental unit,Entire home/apt,...,5100.0,0,0,1,1,0,0,very quiet central top floor with 2 terraces a...,1,0
85,760482,https://www.airbnb.com/rooms/760482,Dúplex-Penthouse Cuzco-Castellana Smart,The flat is very spacious having 100 square me...,Castillejos,Tetuán,40.459890,-3.694720,Entire rental unit,Entire home/apt,...,NaN,0,0,1,1,0,1,dúplex-penthouse cuzco-castellana smart and th...,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24938,1360059324279556027,https://www.airbnb.com/rooms/1360059324279556027,Luxury apartment in Chamartín by beBalmy,Elegant 115 m2 apartment with large terrace an...,Castilla,Chamartín,40.467512,-3.686079,Entire rental unit,Entire home/apt,...,0.0,1,0,1,1,0,1,luxury apartment in chamartín by bebalmy and e...,1,0
25181,1367208187303882310,https://www.airbnb.com/rooms/1367208187303882310,Central Sol residences 5-I,This listing has a strategic location - it wil...,Cortes,Centro,40.415140,-3.700600,Entire rental unit,Entire home/apt,...,0.0,1,0,0,0,0,1,central sol residences 5-i and this listing ha...,1,0
25189,1367400905039791308,https://www.airbnb.com/rooms/1367400905039791308,La Terraza Vista Alegre,Modern 1-Bedroom with Terrace in Vista Alegre ...,Vista Alegre,Carabanchel,40.390433,-3.742443,Entire rental unit,Entire home/apt,...,0.0,1,0,1,0,0,1,la terraza vista alegre and modern 1-bedroom w...,1,0
25272,1368903492306099065,https://www.airbnb.com/rooms/1368903492306099065,Edgar Neville III,"Bathed in natural light, this stylish apartmen...",Cuatro Caminos,Tetuán,40.450625,-3.695489,Entire rental unit,Entire home/apt,...,0.0,1,0,1,0,0,1,"edgar neville iii and bathed in natural light,...",1,0


##### Jardin

In [340]:
df["jardin"] = df["name_description"].apply(lambda x: 1 if "garden" in x else 0)

In [341]:
df["jardin"].value_counts()

jardin
0    16489
1      316
Name: count, dtype: int64